List of datasets:

## Everything

Sebjan.pickle, Kamchatka.pickle -- словари формата

`{
    doc:{    # in a dict, named
        meta:[meta],    # metainfo at the head
        text:[
            {        # sentence (in a list, numbered)
                layer:[
                    morphs    # for mb, ge, ps
                    ],
                layer:''    # e.g. translation
            }
        ]
    }
}`

## Converbs

converbs.pickle -- таблица с конвербами + логлайк


## Morphemes

### Raw

Sebjan_morphemes.pickle, Kamchatka_morphemes.pickle -- raw succession of words like this

` ('ROOT', 'ROOT', 'ROOT'),    # root
 ('-B', '-med', 'v'),
 ('-DEŋ', '-pst.ptc', 'v'),
 ('-E', '-ep', 'v')
 ('END', 'END', 'END')]     # end of the word`

### Single morphemes
 
Sebjan_morphemes_only.pickle, Kamchatka_morphemes_only.pickle -- pd of morphemes with counts only

morpheme_counts.pickle -- pd of morphemes with counts only by two corp together

total_morphemes.pickle -- таблица с морфемами + логлайк


 
 ### Pairs
 
 Sebjan_pairs_dict.pickle, Kamchatka_pairs_dict.pickle -- dicts of (morph): {morph_after: N}
 
 pairs.pickle -- pd with pairs and count only
 
 pairs_by_ll.pickle -- pd with pairs and all the stats
 


In [1]:
import os, re, pickle
from pprint import pprint
import numpy as np
import pandas as pd 

In [107]:
pairs = pd.read_pickle('pairs.pickle')
morphs = pd.read_pickle('morpheme_counts.pickle')

In [108]:
# morphs = pd.read_pickle('total_morphemes.pickle')[['morpheme', 'gloss', 'pos', 's_count', 'k_count']]
# morphs.to_pickle('morpheme_counts.pickle')

In [109]:
pairs['morpheme'] = pairs['1morpheme']
pairs['gloss'] = pairs['1gloss']
pairs['pos'] = pairs['1pos']
del pairs['1morpheme']
del pairs['1gloss']
del pairs['1pos']
pairs = pd.merge(pairs, morphs, on=['morpheme', 'gloss', 'pos'], suffixes=['_pair', '_one'])

In [110]:
pairs.head()

,2gloss,2morpheme,2pos,s_count_pair,k_count_pair,morpheme,gloss,pos,s_count_one,k_count_one
0,END,END,END,1.0,NaN,BIt,1pl.Y,ptl,1.0,0.0
1,END,END,END,5.0,NaN,BIt,1pl.Y,v,5.0,0.0
2,END,END,END,NaN,4.0,(R)U,1pl.ex,?,0.0,4.0
3,END,END,END,NaN,1.0,(R)U,1pl.ex,n,0.0,1.0
4,ptl,tIt,v,NaN,1.0,(R)U,1pl.ex,v,0.0,322.0


In [ ]:
# clean up

pairs['s_count_pair'] = pairs['s_count_pair'].fillna(0)
pairs['k_count_pair'] = pairs['k_count_pair'].fillna(0)
pairs = pairs.loc[pairs['s_count_pair'] != 49803]
pairs = pairs.loc[pairs['gloss'] != 0]
pairs = pairs.loc[pairs['gloss'] != '']
pairs = pairs.loc[pairs['gloss'] != '*']
pairs = pairs.loc[pairs['gloss'] != '***']
pairs = pairs.loc[pairs['gloss'] != '******']
pairs = pairs.loc[pairs['gloss'] != 'ep']

# calculate log-likelihood
# expected = n_of_wds_in_crp_a*(count_in_a+count_in_b)/n_of_wds_in_both
pairs['s_expected'] = pairs['s_count_one']*(pairs['s_count_pair']+pairs['k_count_pair'])/(pairs['s_count_one']+pairs['k_count_one'])
pairs['k_expected'] = pairs['k_count_one']*(pairs['s_count_pair']+pairs['k_count_pair'])/(pairs['s_count_one']+pairs['k_count_one'])
pairs['k_semi_l'] = pairs['k_count_pair']*np.log(pairs['k_count_pair']/pairs['k_expected'])
pairs['s_semi_l'] = pairs['s_count_pair']*np.log(pairs['s_count_pair']/pairs['s_expected'])
pairs['k_semi_l'] = pairs['k_semi_l'].fillna(0)
pairs['s_semi_l'] = pairs['s_semi_l'].fillna(0)
pairs['log_l'] = 2*(pairs['s_semi_l'] + pairs['k_semi_l'])
pairs['ell'] = pairs['log_l']/(82582*np.log(
        pairs['s_expected']*(pairs['s_expected']<pairs['k_expected'])+pairs['k_expected']*(pairs['k_expected']<pairs['s_expected'])
                                     )
                              )
pairs = pairs.sort_values('log_l', ascending=False)

del pairs['k_semi_l']
del pairs['s_semi_l']
del pairs['s_expected']
del pairs['k_expected']

columns = pairs.columns.tolist()
columns = columns[5:8] + columns[:5] + columns[8:]
pairs = pairs[columns]

In [119]:
pairs.head(20)

,morpheme,gloss,pos,2gloss,2morpheme,2pos,s_count_pair,k_count_pair,s_count_one,k_count_one,log_l,ell
2365,R(E),nonfut,v,1pl.in,p,v,384.0,52.0,2084.0,2956.0,415.043799,0.000968
1679,WEːČ,gnr,v,nonfut,R(E),v,19.0,1506.0,335.0,1891.0,358.820077,0.000799
3164,D,prog,v,gnr,WEːČ,v,0.0,458.0,765.0,1675.0,344.585345,0.000840
3285,RI,pst,v,poss.1pl.in,t(I),v,170.0,9.0,1308.0,2712.0,317.468213,0.000946
1681,WEːČ,gnr,v,hab,Gr(E),v,67.0,0.0,335.0,1891.0,253.773354,0.001330
3287,RI,pst,v,poss.1pl.ex,WUn,v,0.0,318.0,1308.0,2712.0,250.326759,0.000653
3154,D,prog,v,sim.cvb,nIkEn,v,112.0,3.0,765.0,1675.0,234.270697,0.000791
3157,D,prog,v,cond.cvb,REk,v,111.0,32.0,765.0,1675.0,129.517872,0.000412
3152,D,prog,v,impf.ptc,RI,v,94.0,21.0,765.0,1675.0,124.531704,0.000421
3148,D,prog,v,inch,L,v,2.0,165.0,765.0,1675.0,107.105180,0.000328


In [89]:
# здесь я находила и убирала дубликаты
# pd.concat(g for _, g in morphs.groupby(['morpheme', 'gloss', 'pos']) if len(g) > 1)

# morphs['k_count'][731] = 134
# morphs['k_count'][74] = 233
# morphs['k_count'][494] = 42
# morphs['k_count'][742] = 39
# morphs = morphs.sort_index()
# morphs = morphs.drop(morphs.index[75]).drop(morphs.index[732]).drop(morphs.index[495]).drop(morphs.index[743])

In [124]:
rel_pairs = pd.concat(g.sort_values('log_l', ascending=False) for _, g in pairs.groupby(['morpheme', 'gloss', 'pos']) if len(g.loc[g['log_l']>10])>0)       

In [126]:
len(rel_pairs['gloss'].unique())

54

In [129]:
rel_pairs = rel_pairs.sort_values('log_l', ascending=False)
rel_pairs['log_l'] = rel_pairs['log_l'].map(lambda x: round(x, 1))
rel_pairs.to_excel('pairs_by_ll.xlsx')